## Robot Learning

## Assignment 3

#### Group names: Edit this cell and write your names here

### Introduction

Consider the following $10 \times 10$ grid world:

<img src="helpers/gridworld_sketch.png" alt="Grid World" title="Grid World" width="650"/>

The agent may start in any cell that is not an obstacle nor the goal.

It can choose between eight actions, which correspond to moving to the directions 

$$a_i \in \{NW,      N,      NE,     E,     SE,    S,     SW,     W\}$$

These are indexed according to the order above, i.e. $a_0 = NW$ and $a_6 = SW$.

The agent must be careful, for the actions are non-deterministic! The agent moves with probability $0.9$ into the desired
direction, but with probability $0.05$ deviates $45^{\circ}$ to the left and with probability $0.05$ deviates $45^{\circ}$ 
to the right of the desired direction due to treacherous gusts unexpectedly sweeping the grid.

The rewards are structured as follows:

* When it reaches a blue cell, it receives a little snack of 10 points.

* All actions entering a white cell receive -1 point.

* When the agent reaches the green goal cell, it receives 100 points and the episode ends.

* When it attempts to enter a red obstacle cell, it receives -20 points and stays in the cell it came from. This does not additionally yield the reward of the cell it came from.

* When it attempts to leave the grid, it receives -30 points and stays in the cell it came from. This does not additionally yield the reward of the cell it came from.

## Task 11

To familiarize yourself with the environment above, answer the following questions:

<div style="text-align: right; font-weight:bold"> 2 + 1 + 2 = 5 Points </div>

* The agent is at $s = (y_s, x_s) = (3, 6)$ and wants to execute $a_5$. What is the probability $P^{a_5}_{s,s'}$ for $s' =(4,7)$ of moving from state $s$ to follow-up state $s'$ when executing the action $a_5$?

* The agent is at $s = (5, 4)$ and wants to execute $a_7$. What is the probability $P^{a_7}_{s,s'}$ for $s'=s$?

* The agent is at $s = (3, 8)$ and wants to execute $a_3$. What is the expected value of the reward?



1)If the agent is on  $s = (y_s, x_s) = (3, 6)$ and wants to execute $a_5$(S), the probability to move from state $s$ to follow-up state $s'$ is:

$P^{a_5}_{s,s'}$= $0.05\times0.05\times0.05\times0.05\times0.9\times0.9\times0.05\times0.05\times0.05\times0.05\times0.05$=$1.58\times 10^{-12}$

The agent goes 4 times to W with the probability 0.05. Then it goes 2 times to S with the probability of 0.9 and then 5 more times to E with the probability of 0.05.

2)If the agent is on  $s = (y_s, x_s) = (5, 4)$ and wants to execute $a_7$ (W), the probaility for $s'$=$s$ is:
$P^{a_7}_{s,s'}$= $Pr\{s_{t+1}=s'|s_{t}=s, a_{t}=a_{7}\}$ + $Pr\{s_{t+1}=s'|s_{t}=s, a_{t}=a_{7}\}$ = 0.05 + 0.05
=0.10 \
To stay on the same cell, the agent must move towards the upper obstacle (4, 4) or the lower one (6, 4).
Since the agent goes to W with the probability 0.9, the probability is 0.05 according to N (upper obstacle) and 0.05 according to S(lower obstacle). Both probabilitys have to be summed up.

3)If the agent is on  $s = (y_s, x_s) = (3, 8)$ and wants to execute $a_3$, the expected value of the reward is:

$R^{a_3}_{s,s'}$=$0.9\times100+0.05\times(-20)+0.05\times(-1)$=90.05
If the agent moves in direction E he will reach the green goal with probability 0.9 (100 points), with probability 0.05 he will reach a obstacle (-20 points) and with probability 0.05 a white cell (-1)


## Task 12

Using the *Iterative Policy Evaluation* Algorithm, compute the value $V^{\pi}(s)$ of all accessible cells $s$ for a policy $\pi(s,a)$ that chooses with probability $0.5$ a random action and otherwise moves to the right.

Intialize $V(s)$ with zero, use a discount parameter of $\gamma=0.9$ and show your results by printing your state values $V^{\pi}(s)$.

<div style="text-align: right; font-weight:bold"> 5 Points </div>

#### Note

For your convenience, you are provided the helper function *getNextStatesRewardsAndProbabilities(state, action)* which returns for a given state $s$ and an action $a$ a list of 3 -tuples of the form

$$[(s_0', R^a_{s,s_0'}, P^a_{s,s_0'}), (s_1', R^a_{s,s_1'}, P^a_{s,s_1'}), \dots]$$

where $s_i'$ are all future states with $P^a_{s,s_i'} \neq 0$. Here $s = (y, x)$ and $s_i' = (y_i', x_i')$ are both tuples of integers, $a \in {0, \dots, 7}$ is an integer, and $R^a_{s,s_i'}$, $P^a_{s,s_i'}$ are both floats.

Also, please find below some data structures which you might find helpful and create code and text cells as necessary to present your solution!

In your implementation, $V(s)$ should be a $10 \times 10$ numpy array and $\pi(s,a)$ should be a $10 \times 10 \times 8$ numpy array, where $\sum_a \pi(s,a) = 1$ for all s!

In [1]:
import numpy as np
from helpers.utils import getNextStatesRewardsAndProbabilities

# this is a list of all states
states = [(y,x) for y in range(10) for x in range(10)]
# this is a list of all states containing obstacles
obstacles = [(1,6), (1,8), (2,1), (2,2), (2,3), (2,4), (2,5), (2,6), (2,8), \
             (3,1), (3,7), (4,3), (4,4), (4,5), (4,6), (5,8), \
             (6,1), (6,2), (6,3), (6,4), (6,5), (6,6), (6,7),\
             (8,8), (8,9), (9,4), (9,8), (9,9)]
# this is a list containing all goal states
terminalStates = [(3,9)]
#this is an array containing all actions
actions = np.array([0, 1, 2, 3, 4, 5, 6, 7]) #[NW,      N,      NE,     E,     SE,    S,     SW,     W]
# example of how to unpack getNextStatesRewardsAndProbabilities(state, action):
# create dummy state and action
s_test = (0,6)
a_test = 3
# how to call helper function and loop over the return values
for sPrime, R, P in getNextStatesRewardsAndProbabilities(state=s_test, action=a_test):
    print('sPrime:', sPrime, 'R:', R, 'P:', P)
    
# once you have state values V, you can print them with okay'ish formatting like so:
#print("State Values:")
#print(np.around(V, 1))

sPrime: (0, 6) R: -30.0 P: 0.05
sPrime: (0, 7) R: 10.0 P: 0.9
sPrime: (1, 7) R: -1.0 P: 0.05


## Task 13

Now it is time to find a good policy. Use the *Policy Iteration* algorithm to compute the optimal value $V^*(s)$ for each accessible cell. Please make sure to apply *Policy Iteration* exhaustively, which means to let policy evaluation converge every time before applying policy improvement.

Retrieve the resulting optimal-policy $\pi^*(s)$. To obtain a greedy policy given $V(s)$, make use of:

$$\pi_{greedy}(s) := \operatorname{argmax}_a Q(s,a) = \operatorname{argmax}_a \sum_{s'}P_{ss'}^a\cdot[R_{ss'}^a+\gamma\cdot V(s')]$$

As implied by these terms, we recommend using intermediate $Q$-values, shaped $10 \times 10 \times 8$ for this step!

Finally, present your results by printing $V^*(s)$ and using our helper function *drawPolicy()* to visualize $\pi^*(s,a)$.

<div style="text-align: right; font-weight:bold"> 5 Points </div>

In [2]:
from helpers.utils import drawPolicy
# print state values with
#print("State Values:")
#print(np.around(V, 1))
# then show policy using helper function as below
# usage of the helper function, where pi is a (10,10,8) numpy array representing a deterministic policy:
# make sure that all entries for pi(s,a) are 0 for all but one action a*, for which pi(s,a*) = 1.0
#drawPolicy(pi)
# this will plot arrows representing your policies into the grid world.

## Task 14

Use the *Value Iteration* algorithm to compute the optimal value $V^*(s)$ for each cell. Make sure to reinitialize $V(s)$ with zero. You can sanity-check your results by comparing with $V^*(s)$ from the previous task.

Finally, present your results by printing $V^*(s)$ and using our helper function *drawPolicy()* to visualize $\pi^*(s,a)$.

<div style="text-align: right; font-weight:bold"> 5 Points </div>

In [3]:
# print state values with
#print("State Values:")
#print(np.around(V, 1))
# then show policy using helper function as below
# usage of the helper function, where pi is a (10,10,8) numpy array representing a deterministic policy:
# make sure that all entries for pi(s,a) are 0 for all but one action a*, for which pi(s,a*) = 1.0
#drawPolicy(pi)
# this will plot arrows representing your policies into the grid world.